![DSME-logo](./utils/DSME_logo.png)

#  Reinforcement Learning and Learning-based Control

<p style="font-size:12pt";> 
<b> Prof. Dr. Sebastian Trimpe, Dr. Friedrich Solowjow </b><br>
<b> Institute for Data Science in Mechanical Engineering (DSME) </b><br>
<a href = "mailto:rllbc@dsme.rwth-aachen.de">rllbc@dsme.rwth-aachen.de</a><br>
</p>

---
Notebook Authors: Lukas Kesper

In this example, we use Dyna-Q to find an optimal policy for the Frozen Lake environment from ["Gymnasium."](https://gymnasium.farama.org/index.html#). An example of the environment is shown below. The goal is to move the elf to the gift without entering one of the ice holes. The state is the position on the grid, and we can move the elf into the same four directions in each state.

<img src="./utils/frozen_lake.gif" alt="Example of Frozen_Lake" width="800">   

Reference: [Reinforcement Learning: An Introduction, by Richard S. Sutton and Andrew G. Barto](http://incompleteideas.net/book/the-book-2nd.html), 

## Library Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import custom_envs
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline
from IPython.display import Video
from IPython.display import display
from screeninfo import get_monitors
from typing import Optional

## Initializations
### Initializing the Agent

Temporal Difference (TD) algorithms are a class of algorithms capable of finding the optimal policy for an environment without needing a model of the environment. In this example, we use the fact, that the action space is independent of the state space. This is a common assumption in reinforcement learning implementations, which allows for a straightforward presentation of the algorithm. However, there are some environments, where this assumption doesn't hold, e.g. the recycling robot from the lecture.

Our `Agent` interacts with the environment to obtain an optimal policy via Dyna-Q. To understand Dyna-Q, consider the following: In standard Q-learning, we only improve the value function. However, we can also use the same experience to improve a model in parallel, which we can in turn use for further refinements of the value function. This is the approach Dyna-Q follows. The algorithm is presented below.

<img src="./utils/dyna-q.png" alt="Dyna-Q algorithm" width="500">   

The training loop can be found in the method `train`. Please note that we call the Q-function `action_value_fn`. Just like in the algorithm above, we use an $\epsilon$-greedy policy via the `epsilon_greedy_policy` method, to ensure exploration. The randomness can be set via the parameter `epsilon`. Additionally, there exists a parameter, `learning_rate`, that is used for updates of the Q-function. 

*Note*: The Agent is similar to the Agent used for Q-learning. However, by adding `self.model` we can use experience for speeding up the training. The parameter `dyn_q_iters` is used to set the rate of replays.

In [ ]:
class Agent():
    def __init__(self, env, gamma=1.0, learning_rate=0.05, epsilon=0.1, dyn_q_iters=100):
        self.env = env
        self.action_value_fn = np.zeros((self.env.observation_space.n, self.env.action_space.n)) # the q-fn
        self.model = np.zeros((self.env.observation_space.n, self.env.action_space.n, 2))
        class StateActionTracker:
            def __init__(self):
                self.states = {}
                self.actions = {}
            def add_state_action_pair(self, state, action):
                if state not in self.states:
                    self.states[state] = 1
                    self.actions[state] = set([action])
                else:
                    self.states[state] += 1
                    if action not in self.actions[state]:
                        self.actions[state].add(action)
            def get_state(self):
                if self.states:
                    return np.random.choice(list(self.states.keys()))
            def get_action(self, state):
                if state in self.actions and self.actions[state]:
                    return np.random.choice(list(self.actions[state]))
        self.visited_states_and_actions = StateActionTracker()
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon # choose 0.0 to make a totally greedy policy, 1.0 for a fully random policy
        self.dyn_q_iters = dyn_q_iters

    def get_random_action(self):
        random_action = np.random.choice(range(self.env.action_space.n))
        return random_action
    def get_best_action(self, state):
        best_action = np.random.choice(np.flatnonzero(np.isclose(self.action_value_fn[state], self.action_value_fn[state].max(),
                                                                 rtol=0.01)))
        return best_action
    def epsilon_greedy_policy(self, state):
        # returns action, choosing a random action with probability epsilon, or the best action
        # regarding Q with probability (1 - epsilon)
        randomly = np.random.random() < self.epsilon
        if randomly:
            action = self.get_random_action()
        else:
            action = self.get_best_action(state)
        return action

    def train(self, num_episodes):
        # Reset environment and pick first action
        for i in range(num_episodes+1):
            state, info = self.env.reset()  # most values are replaced in first env.step(),
            done = False
            while not done:
                # Choose action and perform step
                action = self.epsilon_greedy_policy(state)
                next_state, reward, done, truncated, info = self.env.step(action)
                # TD Update
                best_next_action = self.get_best_action(next_state)
                self.action_value_fn[state][action] = self.action_value_fn[state][action] + self.learning_rate * (reward + self.gamma * self.action_value_fn[next_state][best_next_action] - self.action_value_fn[state][action])
                self.model[state][action] = np.array([next_state, reward]) # Assuming deterministic environment
                self.visited_states_and_actions.add_state_action_pair(state, action)
                state = next_state
                for j in range(self.dyn_q_iters):
                    prev_state = self.visited_states_and_actions.get_state()
                    prev_action = self.visited_states_and_actions.get_action(prev_state)
                    prev_next_state, prev_reward = int(self.model[prev_state][prev_action][0]), self.model[prev_state][prev_action][1]
                    best_next_action = self.get_best_action(prev_next_state)
                    self.action_value_fn[prev_state][prev_action] = self.action_value_fn[prev_state][prev_action] + self.learning_rate * (prev_reward + self.gamma * self.action_value_fn[prev_next_state][best_next_action] - self.action_value_fn[prev_state][prev_action])
            if i % 20 == 0:
                self.visualize(i)

### Preparing the Evaluation

For evaluation, we require methods that visualize the Q-function (`visualize`), as well as a method that is used to evaluate the resulting policy (`evaluate`). We add these methods below.

In [ ]:
def visualize(self, epoch): 
    fig, ax=plt.subplots(figsize=(7,5))
    lines = 4
    rows = 4        
    # Define grid positions:
    pos_x_left = 0.2
    pos_x_mid = 0.5
    pos_x_right = 0.8
    pos_y_up = 0.2
    pos_y_mid = 0.5
    pos_y_down = 0.8
    grid_size = {'x': lines, 'y': rows}
    def gridcreator(pos_x, pos_y):
        grid = []
        for i in range(grid_size['x']):
            for j in range(grid_size['y']):
                x = pos_x + j
                y = pos_y + i
                grid.append((x, y))
        return grid
    top = self.action_value_fn[:,3].reshape((lines, rows))
    top_value_positions = gridcreator(pos_x_mid, pos_y_up)
    right = self.action_value_fn[:,2].reshape((lines, rows))
    right_value_positions = gridcreator(pos_x_right, pos_y_mid)
    bottom = self.action_value_fn[:,1].reshape((lines, rows))
    bottom_value_positions = gridcreator(pos_x_mid, pos_y_down)
    left= self.action_value_fn[:,0].reshape((lines, rows))
    left_value_positions = gridcreator(pos_x_left, pos_y_mid)
    # Define triangles
    ax.set_ylim(lines, 0)
    anchor_points = np.array([[0,0],[0,1],[.5,.5],[1,0],[1,1]]) # Corner coordinates
    corner_indizes = np.array([[0,1,2], [0,2,3],[2,3,4],[1,2,4]]) # Corner indices
    xy_coordinates = np.zeros((lines * rows * 5,2))
    triangles = np.zeros((lines * rows * 4, 3))
    for i in range(lines):
        for j in range(rows):
            k = i*rows+j
            xy_coordinates[k*5:(k+1)*5,:] = np.c_[anchor_points[:,0]+j, 
                                                  anchor_points[:,1]+i]
            triangles[k*4:(k+1)*4,:] = corner_indizes + k*5
    colours = np.c_[left.flatten(), top.flatten(), 
            right.flatten(), bottom.flatten()].flatten()
    ax.triplot(xy_coordinates[:,0], xy_coordinates[:,1], triangles, 
               **{"color":"k", "lw":1})
    tripcolor = ax.tripcolor(xy_coordinates[:,0], xy_coordinates[:,1], triangles, 
                             facecolors=colours, **{"cmap": "coolwarm"})
    ax.margins(0)
    ax.set_aspect("equal")
    fig.colorbar(tripcolor)
    # Define text:
    textsize = 10
    for i, (xi,yi) in enumerate(top_value_positions):
        plt.text(xi,yi,round(top.flatten()[i],2), size=textsize, color="w", 
                 ha='center', va='center')
    for i, (xi,yi) in enumerate(right_value_positions):
        plt.text(xi,yi,round(right.flatten()[i],2), size=textsize, color="w", 
                 ha='center', va='center')
    for i, (xi,yi) in enumerate(left_value_positions):
        plt.text(xi,yi,round(left.flatten()[i],2), size=textsize, color="w", 
                 ha='center', va='center')
    for i, (xi,yi) in enumerate(bottom_value_positions):
        plt.text(xi,yi,round(bottom.flatten()[i],2), size=textsize, color="w", 
                 ha='center', va='center')
    ax.axis('off')
    plt.title("Q-Function, Epoch "+str(epoch))
    # add lines for separation
    for i in range(lines+1):
        x = [0, rows]
        y = [i, i]
        plt.plot(x,y, color='black')
    for i in range(rows+1):
        x = [i, i]
        y = [0, lines]
        plt.plot(x,y, color='black')
    plt.show()

def evaluate(self, env, file, num_runs=5):
    frames = []  # collect rgb_image of agent env interaction
    video_created = False
    for _ in range(num_runs):
        done = False
        obs, info = env.reset()
        out = env.render()
        frames.append(out)
        while not done:
            action = self.get_best_action(obs) 
            obs, reward, done, truncated, info = env.step(action)
            out = env.render()
            frames.append(out)              
    # create animation out of saved frames
    if all(frame is not None for frame in frames):
        fig = plt.figure(figsize=(10, 6))
        plt.axis('off')
        img = plt.imshow(frames[0])
        def animate(index):
            img.set_data(frames[index])
            return [img]
        anim = FuncAnimation(fig, animate, frames=len(frames), interval=20)
        plt.close()
        anim.save(file, writer="ffmpeg", fps=5)

setattr(Agent, 'visualize', visualize)
setattr(Agent, 'evaluate', evaluate)

## Training and Evaluation
Now we train the agent with the algorithm we implemented earlier and evaluate its performance.

### Setting up Agent and Environment
We start by setting up the environment, which is part of our custom environments. We create the environment using `Gymnasium` and a custom map. Here, we invite students to explore how the training changes when the position of holes (H) and Frozen plates (F). Then, we set up an Agent for training. Initially, our Q-function only consists of zeros, but changes during training.

In [ ]:
map = ["SFFH", "FFFH", "HFFH", "HFFG"]
env = gym.make('CustomFrozenLake-v1', render_mode=None, desc=map, is_slippery=False)
env.reset()
agent = Agent(env, gamma=0.9, learning_rate=0.1, epsilon=0.1)

### Training the Agent
*Notice the much lower number of episodes necessary for training compared to Q-learning*

In [ ]:
agent.train(num_episodes=100)

### Evaluation

Above, we can see the evolution of the Q-function during training. Even after 100 epochs, the algorithm should converge to a solution. The coloring already indicates what a greedy policy can look like. Below we show a greedy policy in the environment.

In [ ]:
env = gym.make('CustomFrozenLake-v1', render_mode='rgb_array', desc=map, is_slippery=False)
video = "dyna-q.mp4"
agent.evaluate(env, video, num_runs=5)
Video(video, html_attributes="loop autoplay")